In [1]:
# !pip install -r requirements.txt

In [1]:
import pandas as pd, numpy as np, re, os, base64
import subprocess
import openai, requests, json
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.agents.tools import Tool
from langchain.tools import BaseTool
from langchain.memory import ConversationBufferMemory
from pptx import Presentation 
from pptx.util import Inches
from pptx.enum.text import PP_ALIGN
from pptx.util import Pt
from dotenv import load_dotenv, find_dotenv
import re

In [2]:
_ = load_dotenv("xxxxxx.env") # read local .env file

In [3]:
api_host = 'https://api.stability.ai'
sd_api_key = os.getenv('STABLE_DIFFUSION_API_KEY') # os.getenv('STABLE_DIFFUSION_API_KEY')
engine_id = "stable-diffusion-xl-1024-v1-0"
sd_api =  f"{api_host}/v1/generation/{engine_id}/text-to-image"
headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {sd_api_key}"
            }

In [4]:
#-----------------------------------------------------------------------------------------------------------------------------------------------------
#                                                               CODE FOR LANGCHAIN OPENAI CALLS # pip install langchain, openai
#-----------------------------------------------------------------------------------------------------------------------------------------------------

class ContentGeneration:
    def __init__(self, num_slides, description):
        self.num_slides = num_slides
        self.description = description

    def init_model(self):
        return ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"),
             model = "gpt-3.5-turbo", temperature=0)
    
    def prompt_engg(self):
        template =  """
        Summarize the text delimited by triple backtics\
        in the form of PowerPoint Slides.
        Please summarize the text in the {num_slides} slides.\
        Also Add Title of Each Slide in the Summary in the\
        following format.
        Slide 1 : Title of slide\
        - Pointer 1
        - Pointer 2
        - Pointer 3
        - Pointer 4
        - Pointer 5
        Slide 2 : Title of Slide\
        - Pointer 1
        - Pointer 2
        - Pointer 3
        - Pointer 4
        - Pointer 5
        ```{text}```
        Along the with above summary, Also generate metadata JSON containing \
        number of slides and title of each slides as \
        list within the JSON Keys are 'num_slides','slide_titles'. Also, generate a title for the whole presentation. 
        Generate the number of slides exactly equal to the {num_slides}.
        Give your output in dictionary format. Ex: {{"summary": {{"slide1: title": [pointers],
                           "slide2: title": [pointers],
                            .....}},
              "json_output": <metadata>,
              "title": <title of presentation>}}
        """
        return PromptTemplate(template=template, input_variables=["text","num_slides"])


    def output_parser(self, response):
        if isinstance(response, str):
            response = json.loads(response)
        return response["summary"], response["json_output"], response["title"]
    
    
    def content_generator(self):
        llm = self.init_model()
        prompt = self.prompt_engg()
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        response = llm_chain.run({"text":self.description,"num_slides":self.num_slides})
        return self.output_parser(response)

#--------------------------------------------------------------------------------------------------------------------------------------------------
#                                                            CODE FOR IMAGE PROMPTS
#--------------------------------------------------------------------------------------------------------------------------------------------------

    def image_prompt(self):
        prompt = """
        Summarize each slides pointer in 8-10 words delimited by triple backticks \
        as a prompt for power point background images related to the description and Start with\
        realistic Keyword.\
        Use Following Format:
        Slide 1 Image Prompt: Realistic image as per the context of Slide.  
        Slide 2 Image Prompt: Realistic image as per the context of Slide. 
        ......
        ......
        Please summarize the text in the {num_slides} slides.\
        ```{response}```
        Give your output in a proper python dictionary key/ value pair format. Ex: {{"Slide 1 Image Prompt": "Realistic image as per the context of Slide",
             "Slide 2 Image Prompt": "Realistic image as per the context of Slide",
             ..........: .............,
             ..........: .............}}
        """
        return PromptTemplate(template=prompt, input_variables=["response", "num_slides"])


    def output_img_parser(self, response):
        if isinstance(response, str):
            response = json.loads(response)
        return response


    def image_content(self,response):
        llm = self.init_model()
        prompt = self.image_prompt()
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        response = llm_chain.run({"response":response,"num_slides":self.num_slides})
        print("-------IMAGE CONTENT RESPONSE", response)
        return self.output_img_parser(response)
    

In [5]:
#-----------------------------------------------------------------------------------------------------------------------------------------------------
#                                                               CODE FOR IMAGE GENERATION
#-----------------------------------------------------------------------------------------------------------------------------------------------------

class ImageGeneration:    
    def save_image(self, response,fname):
        print("response_code", response.status_code)
        if response.status_code == 200:
            data = response.json()
            for image in data["artifacts"]:
                with open(f"{fname}.png", "wb") as f:
                    f.write(base64.b64decode(image["base64"]))
    
    def generate_image(self, image_description,fname):
        response = requests.post(
                                f"{api_host}/v1/generation/{engine_id}/text-to-image",
                                headers=headers,
                                json={
                                    "text_prompts": [
                                        {
                                            "text": f"""You will help me generate a realisitic image for a business presentation \
                                                        within the context of an IT company as per the provided text below. \
                                                        Generate a high-resolution image featuring \
                                                        clear objects. Do not include any text or animated elements in the image. \
                                                        Emphasize the realistic appearance of the objects to create a genuine and non-animated visual composition.\
                                                        {image_description}"""
                                        }
                                    ],
                                    "cfg_scale": 14,
                                    "height": 1024,
                                    "width": 1024,
                                    "samples": 1,
                                    "steps": 30,
                                    "seed":10
                                },
                                verify =False
                            )
        return self.save_image(response,fname)


In [6]:
# --------------------------------------------------------------------------------------------------------------------------------------------------------------
#                                                           CODE FOR GENRATING SLIDES    # pip install python-pptx
# --------------------------------------------------------------------------------------------------------------------------------------------------------------

class GeneratePresentation:

    def ppt_obj(self,title):
        ppt = Presentation()
        title_slide_layout = ppt.slide_layouts[0]
        slide = ppt.slides.add_slide(title_slide_layout)
        slide.shapes.title.text = title.upper()  # "Revolutionizing Fraud Detection: ZipAuto's Cloud-Powered Breakthrough in Claims Analysis"
        # slide.shapes.title.left = Inches(0.09)
        # slide.shapes.title.top = Inches(0.05)
        # slide.shapes.title.width = int(ppt.slide_width/2)-Inches(0.08)
        # slide.shapes.title.height = Inches(1)
        title_para = slide.shapes.title.text_frame.paragraphs[0]
        title_para.font.name = "Graphik"
        title_para.font.bold = True
        title_para.font.size = Pt(32)
        # left = int(ppt.slide_width/2)
        # top = Inches(0)
        # pic = slide.shapes.add_picture("./io/img_slide_1.png", left, top, width=int(ppt.slide_width/2), height=ppt.slide_height)
        # # This moves images to the background
        # slide.shapes._spTree.remove(pic._element)
        # slide.shapes._spTree.insert(2, pic._element)
        return ppt

    def gen_slide_content(self,ppt,img,slide_title, pointers_text_list):
        page  = re.sub(r"\D", "", slide_title)
        page = int(page)-1
        blank_slide_layout = ppt.slide_layouts[1]
        slide = ppt.slides.add_slide(blank_slide_layout)
        slide.shapes.title.text = re.sub(r"Slide [0-9]|SLIDE [0-9]|Slide[0-9]|SLIDE[0-9]|:","",slide_title).strip().upper() # "ZipAuto's Challenge"
        slide.shapes.title.left = Inches(0.09)
        slide.shapes.title.top = Inches(0.05)
        slide.shapes.title.width = int(ppt.slide_width/2)-Inches(0.08)
        slide.shapes.title.height = Inches(1)
        title_para = slide.shapes.title.text_frame.paragraphs[0]
        title_para.font.name = "Graphik"
        title_para.font.bold = True
        title_para.font.size = Pt(22)
        title_para.alignment = PP_ALIGN.LEFT
        # left = Inches(1)
        # top = Inches(1)
        # width = Inches(4)
        # height = Inches(3)


        left = int(ppt.slide_width/2)
        top = Inches(0)
        pic = slide.shapes.add_picture(os.path.abspath(img[page]), left, top, width=int(ppt.slide_width/2), height=ppt.slide_height)
        # This moves images to the background
        slide.shapes._spTree.remove(pic._element)
        slide.shapes._spTree.insert(2, pic._element)

        for pointer in pointers_text_list:
            point = slide.shapes.placeholders[1].text_frame.add_paragraph()
            slide.shapes.placeholders[1].left=Inches(0.08)
            slide.shapes.placeholders[1].top=int(ppt.slide_height/6)
            slide.shapes.placeholders[1].width=int(ppt.slide_width/2)-Inches(0.08)
            slide.shapes.placeholders[1].height=Inches(5)
            point.text = pointer.replace("-","")  # "A surge in fraudulent auto claims"
            point.level = 0
            point.font.name = "Graphik"
            point.font.bold = False
            point.font.size = Pt(18)
            point.alignment = PP_ALIGN.LEFT

        return ppt

    
    def convert_pptx_to_pdf(self, input_path, output_path):
        try:
            subprocess.run(["libreoffice", "--headless", "--convert-to", "pdf", input_path])
            print("Conversion completed successfully.")
        except Exception as e:
            print("Conversion failed:", e)
    
    
    def generate_presentation(self,title, content,img,inputFileName = "output.pptx",outputFileName="output.pdf"):
        ppt = self.ppt_obj(title)#"Revolutionizing Fraud Detection: ZipAuto's Cloud-Powered Breakthrough in Claims Analysis")

        for key, val in content.items():
            updated_ppt = self.gen_slide_content(ppt, img,slide_title=key, pointers_text_list=val)

        updated_ppt.save(inputFileName)

        self.convert_pptx_to_pdf(inputFileName, outputFileName)


In [7]:
#--------------------------------------------------------------------------------------------------------------------------------------------------------------
#                                                               CORE LOGIC CALL
#--------------------------------------------------------------------------------------------------------------------------------------------------------------

class Core(ContentGeneration,ImageGeneration,GeneratePresentation):
    def __init__(self, num_slides, description):
        super().__init__(num_slides, description)

    def main(self):
        image_list = []
        content, json_output, title = self.content_generator()
        print("------SLIDE CONTENT--------",content)
        img_content = self.image_content(content)
        print("------Image CONTENT--------", img_content)
        pointer = 1
        for key, val in img_content.items():
            if isinstance(val, list):
                pointers = " ".join(val)
            else:
                pointers = val
            # image_description = key + " " + pointers
            image_description = pointers
            self.generate_image(image_description, fname=f"image_slide_{pointer}")
            image_list.append(f"image_slide_{pointer}.png")
            pointer+=1
            
        with open("output.json","w") as stream:
            json.dump(json_output, stream)
        self.generate_presentation(title, content,image_list)
        return content

In [8]:
if __name__=="__main__":
    df = pd.read_csv("test_cases.csv")
    print(df)
    print(range(df.shape[0]))
    error_case = []
    for row in range(df.shape[0]):
        try:
            print("Case_Study_Name", df['Case_Study_Name'][row])
            if os.path.isdir(os.getcwd()+"/"+df['Case_Study_Name'][row])==True:
                os.chdir(os.getcwd()+"/"+df['Case_Study_Name'][row])
            else:
                os.mkdir(os.getcwd()+"/"+df['Case_Study_Name'][row])
                os.chdir(os.getcwd()+"/"+df['Case_Study_Name'][row])
            print("current_wd", os.getcwd())
            obj = Core(df['Num_slides'][row], df['Description'][row])
            content = obj.main()
            os.chdir('..')
            print("initial_wd_new_case", os.getcwd())
        except Exception as JSONDecodeError:
            try:
                print("try2")
                obj = Core(df['Num_slides'][row], df['Description'][row])
                content = obj.main()
                os.chdir('..')
            except Exception as JSONDecodeError:
                print("Case_Study_Name_Errored", df['Case_Study_Name'][row])
                error_case.append(df['Case_Study_Name'][row])
                os.chdir('..')
                print("errored_wd_case", os.getcwd())
            continue
    print("cases_to_rerun", error_case)

     Case_Study_Name  Num_slides  \
0            ZipAuto           5   
1            Telecom           7   
2          PCL_Costs           6   
3       PCL_Payments           4   
4  Mayank_Handicraft           5   

                                         Description  
0  ZipAuto, a leading insurance provider, encount...  
1  India's telecom market boasts over 1.15 billio...  
2  Power Construction Ltd (PCL) is an engineering...  
3  Power Construction Ltd (PCL) is an engineering...  
4  The Mayank Handicraft is a leading furniture-m...  
range(0, 5)
Case_Study_Name ZipAuto
current_wd /home/jovyan/ZipAuto
try2
------SLIDE CONTENT-------- {'Slide 1: Introduction': ['- ZipAuto encountered a surge in fraudulent auto claims', '- Conventional fraud detection methods were inadequate', '- ZipAuto embarked on harnessing machine learning and deep learning capabilities', '- Objective: develop a cloud-based solution for claims fraud analysis', '- Aim: enhance fraud detection capabilities and im

/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.stability.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


response_code 200


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.stability.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


response_code 200


...

response_code 200


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.stability.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


response_code 200


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.stability.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


response_code 200
convert /home/jovyan/Mayank_Handicraft/output.pptx -> /home/jovyan/Mayank_Handicraft/output.pdf using filter : impress_pdf_Export
Overwriting: /home/jovyan/Mayank_Handicraft/output.pdf
Conversion completed successfully.
initial_wd_new_case /home/jovyan
cases_to_rerun ['PCL_Payments']
